In [1]:
!sudo -H pip3 install -U emoji

Requirement already up-to-date: emoji in /usr/local/lib/python3.5/dist-packages


In [2]:
!sudo -H pip3 install -U emojipy

Requirement already up-to-date: emojipy in /usr/local/lib/python3.5/dist-packages


In [3]:
!sudo -H pip3 install -U reportlab

Requirement already up-to-date: reportlab in /usr/lib/python3/dist-packages


In [4]:
!sudo -H pip3 install -U Python-fontconfig

Requirement already up-to-date: Python-fontconfig in /usr/local/lib/python3.5/dist-packages


In [5]:
import random
import emoji # https://pypi.python.org/pypi/emoji/

# Selection from http://www.webpagefx.com/tools/emoji-cheat-sheet/
# All the choices are picked to have a simple plural
emojis = [':apple:', ':green_apple:', ':tangerine:', ':lemon:', ':watermelon:', ':melon:', ':banana:', ':pear:', ':pineapple:', ':sweet_potato:', ':eggplant:', ':tomato:', ':corn:']

def generate_exercise(upperlimit):
    
    # We want to pick two different numbers
    # from 1 to upperlimit
    l1, l2 = 0, 0
    while l1==l2:
        l1 = random.randint(1,upperlimit)
        l2 = random.randint(1,upperlimit)
        mn, mx = min(l1,l2), max(l1,l2)
    
    # We choose a random emoji character from the list
    # We add the extra space just for visual separation
    emoji_char = random.choice(emojis)+' '
    
    # The code below gets the textual description of the emoji character
    # and also creates its plural form if necessary
    emoji_desc = emoji_char[1:-2]
    if l1 > 1:
        l1_emoji_desc = emoji_desc + 's'
    else:
        l1_emoji_desc = emoji_desc
    if l2 > 1:
        l2_emoji_desc = emoji_desc + 's'
    else:
        l2_emoji_desc = emoji_desc        
    
    # Create the exercise
    message =  "A. " + emoji.emojize(l1*emoji_char, use_aliases=True) + "\n"
    message += "B. " + emoji.emojize(l2*emoji_char, use_aliases=True) + "\n"
    message += "\n"
    message += "Line A has ____ " + l1_emoji_desc +"\n\n"
    message += "Line B has ____ " + l2_emoji_desc +"\n\n"
    choice = random.choice(['more', 'less'])
    if choice == 'more':
        message += str(mx) + " is ____ more than " + str(mn)
    else: 
        message += str(mn) + " is ____ less than " + str(mx)     
    message += "\n------------------------------\n"
    return message


In [6]:
lines = [l  for i in range(3) for l in generate_exercise(10).split("\n")]
lines

['A. 🍠 🍠 🍠 ',
 'B. 🍠 🍠 🍠 🍠 ',
 '',
 'Line A has ____ sweet_potatos',
 '',
 'Line B has ____ sweet_potatos',
 '',
 '4 is ____ more than 3',
 '------------------------------',
 '',
 'A. 🍠 ',
 'B. 🍠 🍠 🍠 🍠 🍠 🍠 ',
 '',
 'Line A has ____ sweet_potato',
 '',
 'Line B has ____ sweet_potatos',
 '',
 '1 is ____ less than 6',
 '------------------------------',
 '',
 'A. 🍋 🍋 🍋 🍋 🍋 🍋 🍋 🍋 🍋 🍋 ',
 'B. 🍋 🍋 🍋 🍋 🍋 🍋 🍋 🍋 ',
 '',
 'Line A has ____ lemons',
 '',
 'Line B has ____ lemons',
 '',
 '10 is ____ more than 8',
 '------------------------------',
 '']

In [7]:
from emojipy import Emoji
# Pdf doesn't need any unicode inside <image>'s alt attribute
Emoji.unicode_alt = False

def replace_with_emoji_pdf(text, size):
    """
    Reportlab's Paragraph doesn't accept normal html <image> tag's attributes
    like 'class', 'alt'. Its a little hack to remove those attrbs
    """

    text = Emoji.to_image(text)
    text = text.replace('class="emojione None"', 'height=%s width=%s' %
                        (size, size))
    return re.sub('alt="'+Emoji.shortcode_regexp+'"', '', text)


In [8]:
!curl -L 'https://github.com/stv0g/unicode-emoji/raw/master/symbola/Symbola.ttf' -o symbola.ttf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   146    0   146    0     0    327      0 --:--:-- --:--:-- --:--:--   328
100 2118k  100 2118k    0     0  2160k      0 --:--:-- --:--:-- --:--:-- 16.0M


In [9]:
from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.pdfgen import canvas
from reportlab.rl_config import defaultPageSize
from reportlab.pdfbase import pdfmetrics
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY

import re

import fontconfig




def text2pdf(lines, filename, font_name):
    
    
    # Register font
    fonts = fontconfig.query(family=font_name, lang='en')
    font_file = fonts[0].file 
    registered_font = TTFont(font_name, font_file)
    pdfmetrics.registerFont(registered_font)

    # width, height = defaultPageSize
    
    styles = getSampleStyleSheet()
    style = styles["Normal"]
    style.fontName = font_name
    
    doc = SimpleDocTemplate(filename,pagesize=letter,
                        rightMargin=72,leftMargin=72,
                        topMargin=72,bottomMargin=18)

    Story = []
    for line in lines:
        content = replace_with_emoji_pdf(Emoji.to_image(line), style.fontSize)
        para = Paragraph(content, style)
        Story.append(para)
        Story.append(Spacer(1, 12))
 
    doc.build(Story)


In [10]:
text2pdf(lines, "math.pdf", "Ubuntu")